In [12]:
import argparse
import struct
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad

#Define constants
CHUNK_SIZE = 1000
BLOCK_SIZE = 16

#     # Load firmware binary from infile
#     with open(infile, 'rb') as fp:
#         firmware = fp.read()

#Load test_firmware Binary
with open("test_firmware.bin", 'rb') as fp:
    firmware = fp.read()
    
message = "YEEEEEEEEEEEEEEEEEEEEET"

version = 3

print(len(firmware))

# Create variable to hold message
msg = message.encode() + b'\00'

# Pack version and size into two little-endian shorts
# metadata = struct.pack('<HH', version, len(firmware))

#Split firmware into 1K chunks

chunks_needed = int(len(firmware)/CHUNK_SIZE)

chunks=list()

for i in range(chunks_needed):
    print(i)
    chunks.append(firmware[i * CHUNK_SIZE:(i + 1) * CHUNK_SIZE])

if(CHUNK_SIZE * (chunks_needed) - len(firmware) != 0):
    chunks.append(firmware[CHUNK_SIZE * (chunks_needed):])

#Encrypt each chunk

#Initialize AES

key = get_random_bytes(16) #For Testing Purposes only

#Encrypt each Chunk with AES

final_output = b'' #Final output?


for i, chunk in enumerate(chunks):
    
    #Set up AES Cipher
    aes_cipher = AES.new(key, AES.MODE_GCM)
    #Set up metadata
    metadata = struct.pack('<hhhh', len(pad(chunk, BLOCK_SIZE)), version, len(firmware), i)
    aes_cipher.update(metadata)
    
    #Get Cipher Text
    ciphertext, tag = aes_cipher.encrypt_and_digest(pad(chunk, BLOCK_SIZE))
    
    #THings for testing :D
#     print(metadata)
#     print(ciphertext)
#     print(len(ciphertext))
#     print(tag)
#     print(len(tag))

    #Add result to final output
    final_output += (metadata + aes_cipher.nonce + tag + ciphertext)

# Add release message

aes_cipher = AES.new(key, AES.MODE_GCM)
#Set up metadata
metadata = struct.pack('<hhhh', len(pad(msg, BLOCK_SIZE)), version, len(firmware), -1)
aes_cipher.update(metadata)
#Get Cipher Text
ciphertext, tag = aes_cipher.encrypt_and_digest(pad(chunk, BLOCK_SIZE))
final_output += (metadata + aes_cipher.nonce + tag + ciphertext)

#More Testing
print(final_output)

# Write firmware blob to outfile
# with open(outfile, 'wb+') as outfile:
#     outfile.write(firmware_blob)


2826
0
1
b'\xf0\x03\x03\x00\n\x0b\x00\x00\x95\xda\xbc^\x84\x8f6\xe0Yq1=2\xbd\xfa\x96\xe5\xac1\xee\xd3\x93\x13\xe5\xea\x04.C;\x84.\x94\xea\'\x14Y\xf2u\xbc\x8b3C\xe6\xba\x0e0\x1ei\x94\xa0\xaaO.\xba\x08|\xf1F\xbf\xbf\xe9\xe0\x1b/\x96.\xdd?V_\x15}\xe1\xe6\'M\x9e\x80[T\x08a\x8e\xe5{\xa7W\x82\x9c\xfb\x05J\xa9\\\xab\\\x96tF\xddq\x15\xbe\xc8\x02}R\x06\xc9\xdeU+\x1b\x1d\xed\xe7a\x08^\xfa\x90qa\x88\xf3uT\xcc\xcc"]4\xc9\xd7\x0b\xf2i\xe0\xaf@}\xf1\x0b\x04\xf4\xc87\x83\xde9\xa3\xf9\x03\xf5g\x13\x01\x14xS\x04\xf3\x1a\xe7kY\x87lU\x97\xa1l\x89\xd5\x9a\xcc~\\\x1d\nJ\xebt\t\xf3\x00<\xd0\xca\x0f\x17a^(\x06\xa6\x1d\x0es\x117\x1b\xadi\x08VM\x17\x0f\xf4\xdcl\x06G\xd9q\xb9\xbc\x81U,\\\xcb\x1a\\\xb8\xc6\x13-<\xf3\x00\x17\xfd\xac\xb8\xa9\x00\nBI\xaa\xefN\xb6\x93\x00ZuI\x98\xbci\x88\x17\xc6\x93\xa9cG\x81\x89\x1e\x00c\x02\xee\xd8\x14\xac\xb9(\x85\xc1\x8d\xa3\xda\xf3k\xdf]\xa7G%WJ\xe2r\x0e\xechH7\t\xee\xae\xde\x8d\xd6\x18\x93\x13\x80\xb2m\x0cT\xa5\x8b{b\x1f\x07\xa5\xb9\x90OE^\x10\xfe\xcc\xe0\x1df\x18\xbah\xfa\xe5